In [1]:
%load_ext autoreload
%autoreload 2

In [59]:
from importlib import reload
from api.gcp import connect_cloud
import pandas as pd
import pymysql
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table, Column, MetaData, Integer, String, ForeignKey, Float, create_engine, insert,BigInteger
cloud_utility = connect_cloud.CloudUtility()

In [3]:
port = "5432"
username = 
password = 
host = 
database = "postgres"

engine = create_engine(f"postgresql+pg8000://{username}:{password}@{host}:{port}/{database}", echo=True)
session = sessionmaker(bind=engine)

In [63]:
e = engine.connect()

In [ ]:
metadata_obj = MetaData()

table = Table('historical_ticker_data', metadata_obj,
    Column('symbol', String, primary_key=True),
    Column('time', BigInteger, primary_key=True),
    Column('vol_weighted_price', Float),
    Column('transacs', Integer),
    Column('vol', Integer),
    Column('open', Float),
    Column('high', Float),
    Column('low', Float),
    Column('close', Float))

metadata_obj.bind = engine
metadata_obj.create_all(engine)

In [9]:
df = cloud_utility.read_file_from_gcs("tickers/historical_ticks_5", num_workers=50)

In [36]:
df = df.fillna(0)

In [38]:
df.v = df.v.astype(int)
df.n = df.n.astype(int)
df.t = df.t.astype(int)
df.vw = df.vw.astype(float)
df.o = df.o.astype(float)
df.h = df.h.astype(float)
df.l = df.l.astype(float)
df.c = df.c.astype(float)
df.symbol = df.symbol.astype(str)

In [40]:
df_list = np.array_split(df, 50)

In [43]:
df_entries = []

for chunk in df_list: 
    df_entries.append([{'symbol': row.symbol, 
          'time': row.t, 
          'vol_weighted_price': row.vw,
          'transacs': row.n,
          'vol': row.v,
          'o': row.o,
          'h': row.h,
          'l': row.l,
          'c': row.c,
        } for row in chunk.itertuples()])

In [ ]:
t = e.execute("SELECT * FROM historical_ticker_data")